# Load the Query Paraphrasing Dataset

In [1]:
import json
import pandas as pd

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

# Load the dataset
dev_data = load_jsonl("/content/dev.jsonl")
train_data = load_jsonl("/content/train.jsonl")
test_data = load_jsonl("/content/test.jsonl")

dev_data.head()

,q1,q2,label,category
0,چه زمانی باید گوشی را شارژ کنیم؟,چه زمانی باید گوشی را به شارژ زد؟,1,natural
1,با چه چیزی دیوار را تمیز کنیم؟,با چه چیزی گیتار را تمیز کنیم؟,0,natural
2,چه چیزی مردم را از Quora دور می کند؟,"آیا راهی برای خاموش کردن گزینه ""دعوت از مردم ب...",0,qqp
3,کتابهایی که توسط aiims topper ۲۰۱۶ مورد مطالعه...,برای ورود PG من در AIIMS چه کتابهایی باید مطال...,0,qqp
4,کدام کشور ها به ببرهای اقتصادی آسیا لقب گرفته ...,گلزار به کدام کشور ها سفر کرده است؟,0,natural


In [2]:
! pip install transformers datasets pandas scikit-learn bitsandbytes accelerate

# Convert to Train and Test Examples

In [3]:
train_data_ex = []
for _, row in train_data.iterrows():
    train_data_ex.append({'question1': row['q1'], 'question2': row['q2'], 'label': row['label']})

test_data_ex = []
for _, row in test_data.iterrows():
    test_data_ex.append({'question1': row['q1'], 'question2': row['q2'], 'label': row['label']})

# Print first few examples to verify
print(train_data_ex[:5])
print(test_data_ex[:5])

[{'question1': 'چگونه می توانم کم کم وزن خود را کاهش دهم؟', 'question2': 'چگونه وزن کم کنم؟', 'label': '1'}, {'question1': 'چگونه استمناء بر قدرت تمرکز شخص تأثیر می گذارد؟', 'question2': 'آیا چرک روی لوزه ها می تواند نشانه ای از STD باشد؟', 'label': '0'}, {'question1': 'وقتی خواب کسی را دیدم و گفتم آنها در حال مرگ هستند ، چه معنایی دارد؟', 'question2': 'وقتی خواب می بینم که کسی در حال مرگ باشد معنی اش چیست؟', 'label': '0'}, {'question1': 'چگونه می توانم فایل های apk را تغییر دهم؟', 'question2': 'بهترین روش برای ویرایش فایل های APK چیست؟', 'label': '1'}, {'question1': 'از کجا می توانم کفش های کپی درجه اول را در بمبئی بخرم؟', 'question2': 'از کجا ، در بنگلور می توانید کفش های درجه اول را پیدا کنید؟', 'label': '0'}]
[{'question1': 'آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنترل کننده این قلمرو روحانی کیست؟', 'question2': 'چه چیزی روح فرد را می شکند؟', 'label': '0'}, {'question1': 'چگونه می توانم تماشای فیلم های پورنو را متوقف کنم؟', 'question2': 'برای جلوگیری از تماشای کامل پورنو باید چه

# Load the Models with the Specified Configuration

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig

# Define the model loading function
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    model = AutoModelForSequenceClassification.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")
    return model, tokenizer


In [5]:
persian_mind_model, persian_mind_tokenizer = load_model("universitytehran/PersianMind-v1.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/688k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.50M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at universitytehran/PersianMind-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import pipeline

model_pipeline = pipeline("text-classification", model=persian_mind_model, tokenizer=persian_mind_tokenizer)

# Zero Shot

In [7]:
def zero_shot_evaluate(model_pipeline, evaluation_data):
    predicted_labels = []
    true_labels = []
    test_examples = []

    for data_point in evaluation_data:
        q1 = data_point['question1']
        q2 = data_point['question2']
        true_label = data_point['label']

        input_prompt = f"آیا این دو سوال مترادف هستند?\n سوال 1: {q1}\n سوال 2: {q2}"
        output = model_pipeline(input_prompt)

        pred_label = 1 if output[0]['label'] == 'LABEL_1' else 0
        predicted_labels.append(pred_label)
        true_labels.append(int(true_label))
        test_examples.append((q1, q2, int(true_label), pred_label))

    return predicted_labels, true_labels, test_examples

In [8]:
preds, labls, inputs = zero_shot_evaluate(model_pipeline, test_data_ex[:50])

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [9]:
from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(labls, preds)
f1 = f1_score(labls, preds)

print(f" Accuracy: {accuracy}, F1 Score: {f1}")

 Accuracy: 0.36, F1 Score: 0.40740740740740744


In [10]:
# Print some examples
print("\nSome examples with their predicted and true labels:")
for i in range(5):  # print first 5 examples
    q1, q2, true_label = test_data_ex[i]
    print(f"Question 1: {test_data_ex[i]['question1']}")
    print(f"Question 2: {test_data_ex[i]['question2']}")
    print(f"True Label: {test_data_ex[i]['label']}, Predicted Label: {preds[i]}")
    print("-----")


Some examples with their predicted and true labels:
Question 1: آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنترل کننده این قلمرو روحانی کیست؟
Question 2: چه چیزی روح فرد را می شکند؟
True Label: 0, Predicted Label: 0
-----
Question 1: چگونه می توانم تماشای فیلم های پورنو را متوقف کنم؟
Question 2: برای جلوگیری از تماشای کامل پورنو باید چه کاری انجام دهم؟
True Label: 1, Predicted Label: 0
-----
Question 1: چه کسانی امام علی را خدا میدانند؟
Question 2: چه کسانی می توانند امام زمان را ببینند؟
True Label: 0, Predicted Label: 1
-----
Question 1: آیا قرار است دونالد ترامپ رئیس جمهور بعدی ایالات متحده باشد؟
Question 2: شانس اینکه دونالد ترامپ رئیس جمهور بعدی آمریکا باشد ، چیست؟
True Label: 1, Predicted Label: 1
-----
Question 1: چگونه می توانم سوالی را در این باره بپرسم؟
Question 2: چگونه می توانم سوال بپرسم؟
True Label: 0, Predicted Label: 0
-----


# N_Shots Function

In [11]:
def few_shot_evaluate(model_pipeline, evaluation_data, n_shots, shot_examples):
    predicted_labels = []
    true_labels = []
    few_shot_test_examples = []

    # Create the few-shot prompt
    few_shot_prompt = "تشخیص تو درمورد مترادف بودن دو سوال داده شده با توجه به مثال‌های زیر چیست؟\n"
    few_shot_prompt += "مثال:\n"
    for shot in shot_examples:
        few_shot_prompt += f"سوال 1: {shot['question1']}\nسوال 2: {shot['question2']}\nجواب: {'بله' if shot['label'] == 1 else 'خیر'}\n"
    few_shot_prompt += "\n"

    # Evaluate each test example
    for data_point in evaluation_data:
        q1 = data_point['question1']
        q2 = data_point['question2']
        true_label = data_point['label']

        # Create the full prompt by appending the current example to the few-shot prompt
        input_prompt = few_shot_prompt + f"سوال 1: {q1}\nسوال 2: {q2}"

        # Get the model's prediction
        output = model_pipeline(input_prompt)
        pred_label = 1 if output[0]['label'] == 'LABEL_1' else 0

        # Store the results
        predicted_labels.append(pred_label)
        true_labels.append(int(true_label))
        few_shot_test_examples.append((q1, q2, int(true_label), pred_label))

    return predicted_labels, true_labels, few_shot_test_examples


# Five Shots

In [12]:
# Define the number of shots and select shot examples from the training data
n_shots = 5
shot_examples = train_data_ex[:n_shots]

# Evaluate the model using the few-shot approach
preds, labls, inputs =few_shot_evaluate(model_pipeline, test_data_ex[:50], n_shots, shot_examples)

In [13]:
# Calculate accuracy and F1 score
accuracy = accuracy_score(labls, preds)
f1 = f1_score(labls, preds)

# Print the results
print(f"Five-shots Learning - Accuracy: {accuracy}, F1 Score: {f1}")

# Print some examples
print("\nSome examples with their predicted and true labels:")
for i in range(5):  # print first 5 examples
    q1, q2, true_label = test_data_ex[i]
    print(f"Question 1: {test_data_ex[i]['question1']}")
    print(f"Question 2: {test_data_ex[i]['question2']}")
    print(f"True Label: {test_data_ex[i]['label']}, Predicted Label: {preds[i]}")
    print("-----")

Five-shots Learning - Accuracy: 0.4, F1 Score: 0.4642857142857143

Some examples with their predicted and true labels:
Question 1: آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنترل کننده این قلمرو روحانی کیست؟
Question 2: چه چیزی روح فرد را می شکند؟
True Label: 0, Predicted Label: 0
-----
Question 1: چگونه می توانم تماشای فیلم های پورنو را متوقف کنم؟
Question 2: برای جلوگیری از تماشای کامل پورنو باید چه کاری انجام دهم؟
True Label: 1, Predicted Label: 1
-----
Question 1: چه کسانی امام علی را خدا میدانند؟
Question 2: چه کسانی می توانند امام زمان را ببینند؟
True Label: 0, Predicted Label: 1
-----
Question 1: آیا قرار است دونالد ترامپ رئیس جمهور بعدی ایالات متحده باشد؟
Question 2: شانس اینکه دونالد ترامپ رئیس جمهور بعدی آمریکا باشد ، چیست؟
True Label: 1, Predicted Label: 1
-----
Question 1: چگونه می توانم سوالی را در این باره بپرسم؟
Question 2: چگونه می توانم سوال بپرسم؟
True Label: 0, Predicted Label: 0
-----


# One Shot

In [14]:
# Define the number of shots and select shot examples from the training data
n_shots = 1
shot_examples = train_data_ex[:n_shots]

# Evaluate the model using the few-shot approach
preds, labls, inputs =few_shot_evaluate(model_pipeline, test_data_ex[:50], n_shots, shot_examples)

In [15]:
# Calculate accuracy and F1 score
accuracy = accuracy_score(labls, preds)
f1 = f1_score(labls, preds)

# Print the results
print(f"One-shot Learning - Accuracy: {accuracy}, F1 Score: {f1}")

# Print some examples
print("\nSome examples with their predicted and true labels:")
for i in range(5):  # print first 5 examples
    q1, q2, true_label = test_data_ex[i]
    print(f"Question 1: {test_data_ex[i]['question1']}")
    print(f"Question 2: {test_data_ex[i]['question2']}")
    print(f"True Label: {test_data_ex[i]['label']}, Predicted Label: {preds[i]}")
    print("-----")

One-shot Learning - Accuracy: 0.36, F1 Score: 0.42857142857142855

Some examples with their predicted and true labels:
Question 1: آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنترل کننده این قلمرو روحانی کیست؟
Question 2: چه چیزی روح فرد را می شکند؟
True Label: 0, Predicted Label: 0
-----
Question 1: چگونه می توانم تماشای فیلم های پورنو را متوقف کنم؟
Question 2: برای جلوگیری از تماشای کامل پورنو باید چه کاری انجام دهم؟
True Label: 1, Predicted Label: 0
-----
Question 1: چه کسانی امام علی را خدا میدانند؟
Question 2: چه کسانی می توانند امام زمان را ببینند؟
True Label: 0, Predicted Label: 1
-----
Question 1: آیا قرار است دونالد ترامپ رئیس جمهور بعدی ایالات متحده باشد؟
Question 2: شانس اینکه دونالد ترامپ رئیس جمهور بعدی آمریکا باشد ، چیست؟
True Label: 1, Predicted Label: 1
-----
Question 1: چگونه می توانم سوالی را در این باره بپرسم؟
Question 2: چگونه می توانم سوال بپرسم؟
True Label: 0, Predicted Label: 1
-----
